# Azure AI Search with Hugging Face embedding models

This code demonstrates how to use Azure AI Search with a Hugging Face embedding model, [E5-small-v2](https://huggingface.co/intfloat/e5-small-v2) and the Azure AI Search Documents Python SDK.

Azure AI Search on any tier supports vector workloads, but we recommend Basic or higher for this demo. [Enable semantic ranker](https://learn.microsoft.com/azure/search/semantic-how-to-enable-disable) if you want to run the hybrid semantic query example at the end of this notebook.

### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

### Install packages

In [ ]:
! pip install -r azure-search-vector-python-huggingface-model-sample-requirements.txt --quiet

### Load .env file

Copy `.env-sample` to an `.env` file in the sample folder, and update accordingly. The search service must exist, but the search index is created and loaded during code execution. Provide a unique name for the index. Endpoint and API key can be found in the Azure portal.

In [ ]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]) if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else DefaultAzureCredential()
index_name = os.environ["AZURE_SEARCH_INDEX"]

### Create embeddings

Reads the local text-sample.json file, generates embeddings using the pre-trained E5-small-V2 embeddings model, and exports the vectorized output to a local file that can be consumed during indexing.

In [ ]:
from sentence_transformers import SentenceTransformer  
import os
import json

model = SentenceTransformer('intfloat/e5-small-v2')  
sample_path = os.path.join("..", "..", "data", "text-sample.json")
with open(sample_path, 'r', encoding='utf-8') as file:  
    data = json.load(file)  
  
for item in data:  
    title = item['title']  
    content = item['content']  
    title_embeddings = model.encode(title, normalize_embeddings=True)  
    content_embeddings = model.encode(content, normalize_embeddings=True)  
    item['titleVector'] = title_embeddings.tolist()  
    item['contentVector'] = content_embeddings.tolist()  

output_path = os.path.join("..", "..", "data", "docVectors-e5.json")
with open(output_path, "w") as f:  
    json.dump(data, f)  


### Create a search index

Create a search index schema with a vector search configuration.

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticSearch,
    SemanticField,
    SemanticPrioritizedFields,
    SearchIndex
)

# Create a search index  
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)  
fields = [  
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),  
    SearchableField(name="title", type=SearchFieldDataType.String),  
    SearchableField(name="content", type=SearchFieldDataType.String),  
    SearchableField(name="category", type=SearchFieldDataType.String, filterable=True),  
    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),  
                searchable=True, vector_search_dimensions=384, vector_search_profile_name="myHnswProfile"),  
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),  
                searchable=True, vector_search_dimensions=384, vector_search_profile_name="myHnswProfile"),  
]  
  
# Configure the vector search configuration  
vector_search = VectorSearch(  
    algorithms=[  
        HnswAlgorithmConfiguration(  
            name="myHnsw"
        ),  
    ],  
    profiles=[  
        VectorSearchProfile(  
            name="myHnswProfile",  
            algorithm_configuration_name="myHnsw",   
        ),  
    ],  
)  
  
semantic_config = SemanticConfiguration(  
    name="my-semantic-config",  
    prioritized_fields=SemanticPrioritizedFields(  
        title_field=SemanticField(field_name="title"),  
        keywords_fields=[SemanticField(field_name="category")],  
        content_fields=[SemanticField(field_name="content")]  
    )  
)  
  
# Create the semantic settings with the configuration  
semantic_search = SemanticSearch(configurations=[semantic_config])  
  
# Create the search index with the semantic settings  
index = SearchIndex(name=index_name, fields=fields,  
                    vector_search=vector_search, semantic_search=semantic_search)  
result = index_client.create_or_update_index(index)  
print(f'{result.name} created')  


### Upload and store embeddings

This step uploads the JSON document containing your embeddings and sends it to a search client for indexing.

In [ ]:
from azure.search.documents import SearchClient
import json

# Upload some documents to the index
output_path = os.path.join("..", "..", "data", "docVectors-e5.json")
with open(output_path, 'r') as file:  
    documents = json.load(file)  
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)  
print(f"Uploaded {len(documents)} documents") 

### Perform a vector similarity search

In [ ]:
from azure.search.documents.models import VectorizedQuery
from sentence_transformers import SentenceTransformer  


model = SentenceTransformer('intfloat/e5-small-v2')  
# Pure Vector Search  
query = "tools for software development"  
  
search_client = SearchClient(endpoint, index_name, credential=credential)  
query_embeddings = model.encode(query, normalize_embeddings=True)  
vector_query = VectorizedQuery(vector=query_embeddings.tolist(), k_nearest_neighbors=1, fields="contentVector")
  
results = search_client.search(  
    search_text=None,
    vector_queries= [vector_query], 
    select=["title", "content", "category"],  
    top=1
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


### Perform a cross-field vector search

In [ ]:
# Pure Vector Search  
query = "tools for software development"  
  
query_embeddings = model.encode(query, normalize_embeddings=True)  
vector_query = VectorizedQuery(vector=query_embeddings.tolist(), k_nearest_neighbors=1, fields="contentVector, titleVector")
  
results = search_client.search(  
    search_text=None,
    vector_queries= [vector_query], 
    select=["title", "content", "category"],
    top=1
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


### Perform a multi-vector search

In [ ]:
# Multi-Vector Search  
query = "tools for software development"  
  
query_embeddings = model.encode(query, normalize_embeddings=True)  
vector_query_1 = VectorizedQuery(vector=query_embeddings.tolist(), k_nearest_neighbors=1, fields="titleVector")
vector_query_2 = VectorizedQuery(vector=query_embeddings.tolist(), k_nearest_neighbors=1, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries=[vector_query_1, vector_query_2],  
    select=["title", "content", "category"],
    top=1
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


### Perform a pure vector search with a filter

In [ ]:
# Pure Vector Search  
query = "tools for software development"  
  
query_embeddings = model.encode(query, normalize_embeddings=True)  
vector_query = VectorizedQuery(vector=query_embeddings.tolist(), k_nearest_neighbors=1, fields="contentVector")
  
results = search_client.search(  
    search_text=None,
    vector_queries= [vector_query], 
    filter="category eq 'Developer Tools'",
    select=["title", "content", "category"],  
    top=1
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  



### Perform a hybrid search

In [ ]:
# Pure Vector Search  
query = "scalable storage solution"  
  
query_embeddings = model.encode(query, normalize_embeddings=True)  
vector_query = VectorizedQuery(vector=query_embeddings.tolist(), k_nearest_neighbors=1, fields="contentVector")
  
results = search_client.search(  
    search_text=query,
    vector_queries= [vector_query], 
    select=["title", "content", "category"],  
    top=1
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


### Perform a semantic hybrid search

In [ ]:
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType

# Semantic Hybrid Search
query = "what is azure search?"

query_embeddings = model.encode(query, normalize_embeddings=True) 
vector_query = VectorizedQuery(vector=query_embeddings.tolist(), k_nearest_neighbors=1, fields="contentVector")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    query_type=QueryType.SEMANTIC, semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
    top=1
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")
